In [1]:
from transformers import pipeline
from tqdm import tqdm

import json
import os
import re

2023-12-05 08:02:28.810552: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 08:02:28.814732: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 08:02:28.863792: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 08:02:28.863842: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 08:02:28.863875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
unmasker = pipeline('fill-mask', model='xlm-roberta-base')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def load_json(file, path='../NumEval - Task 3/'):
    with open(os.path.join(path, file), 'r') as json_file:
        data = json.load(json_file)
    return data

'''
read train and dev files
'''
train_data = load_json('Train_Numerical_Reasoning.json')
dev_data = load_json('Dev_Numerical_Reasoning.json')

In [6]:
def remove_date(text):
    # Define a regular expression pattern to match the date part within brackets
    date_pattern = r'\([^)]*\)'
    date_pattern = r'^\([^)]+\)\s*'
    # Use re.sub to replace the matched pattern with an empty string
    cleaned_text = re.sub(date_pattern, '', text)

    return cleaned_text.strip()

In [7]:
def process_data(sample, num_gt, replace_token='mask', task='train'):
    '''
    teacher forcing only during training, hence reasoning prompt would be prepended only to the train samples
    '''
    
    news = sample['news']
    masked_headline = sample['masked headline']
    calculation = sample['calculation']
    ans = str(sample['ans'])
    num_gt.append(ans)
    
    prompt = news + " " + masked_headline.replace('____', '<mask>')
    return remove_date(prompt)

In [42]:
train_num_gt = []
dev_num_gt = []
processed_train = [process_data(sample, train_num_gt) for sample in train_data]
processed_dev = [process_data(sample, dev_num_gt) for sample in dev_data]

In [39]:
'''
zero-shot evaluation
'''
dev_num_preds = []
exceptions = []

for i, prompt in tqdm(enumerate(processed_dev)):
    try:
        val = unmasker(prompt)
        flag=0
        for res in val:
            if res['token_str'].isnumeric():
                dev_num_preds.append(res['token_str'])
                flag=1
                break
        if flag==0:
            dev_num_preds.append(val[0]['token_str'])
    except Exception as error:
        print(f"Error raised at i = {i}; Continue")
        exceptions.append(i)
        continue

1171it [03:13,  7.21it/s]

Error raised at i = 1169; Continue


1187it [03:15,  6.51it/s]

Error raised at i = 1187; Continue


2572it [07:25,  5.77it/s]


In [41]:
len(dev_num_preds)

2570

In [44]:
del dev_num_gt[1169]

In [45]:
del dev_num_gt[1187]

In [46]:
len(dev_num_gt)

2570

In [22]:
from sklearn.metrics import accuracy_score

In [47]:
accuracy_score(dev_num_preds, dev_num_gt)

0.58715953307393